# L1 Language Models, the Chat Format and Tokens

## Configuration
#### Chargement de la clé API et des librairies nécessaires

In [3]:
import tiktoken
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)


In [4]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0,
    )
    return response.choices[0].message.content

**Note**: Tous les notebooks suivants utilisent la version `0.27.0` de la bibliothèque d'OpenAI. 

Pour utiliser la version `1.0.0`, il faut utiliser la version suivante : 

```python
client = openai.OpenAI()

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content
```

## Interroger le modèle et obtenir la réponse

In [5]:
response = get_completion("Quelle est la capitale de la France ?")

In [6]:
print(response)

La capitale de la France est Paris.


## Tokens

In [7]:
response = get_completion("Prenez les lettres de 'lollipop' et inversez-les.")
print(response)

En inversant les lettres de 'lollipop', on obtient 'popillol'.


"lollipop" in reverse should be "popillol"

In [10]:
response = get_completion("""Prenez les lettres de
l-o-l-l-i-p-o-p et inversez-les.""")

In [11]:
response

'En inversant les lettres de "l-o-l-l-i-p-o-p", on obtient "p-o-p-i-l-l-o-l".'

## Fonction helper (chat format)

In [12]:
def get_completion_from_messages(messages, 
                                 model="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message.content

In [13]:
messages =  [  
{'role':'system', 
 'content':"""Vous êtes un assistant qui réponds en prenant le style d'un conteur pour enfants."""},    
{'role':'user', 
 'content':"""Ecrivez une histoire sur une carotte heureuse."""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Il était une fois, dans un jardin enchanté, une carotte joyeuse nommée Carla. Carla avait une belle robe orange, brillante sous les doux rayons du soleil, et un feuillage vert émeraude qui dansait au gré du vent. Chaque matin, elle se réveillait en voyant les autres légumes autour d'elle, tous vêtus de leurs plus belles couleurs. Mais Carla, elle, avait une particularité : elle chantait dès le lever du soleil !

« La la la, je suis une carotte en or ! » chantait-elle gaiement, ce qui attirait les abeilles, les papillons et même les oiseaux qui venaient écouter ses mélodies.

Tous les jours, Carla rêvait de voyages au-delà du potager. Elle avait entendu les histoires racontées par un vieux radis sage qui parlait d'une ville où les légumes parsemaient les rues et où les gens les adoraient. "Oh, comme j'aimerais y aller !" se disait-elle en sautillant joyeusement.

Un jour, alors qu'elle profitait du doux parfum des fleurs, elle rencontra un petit lapin nommé Léon. Elle lui confia son rêv

In [14]:
# length
messages =  [  
{'role':'system',
 'content':'Toutes vos réponses doivent être d"une seule phrase.'},    
{'role':'user',
 'content':'Ecrivez une histoire sur une carotte heureuse.'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Dans un jardin ensoleillé, une carotte nommée Carotina grandissait joyeusement, rêvant de devenir la star des salades.


In [15]:
# combined
messages =  [  
{'role':'system',
 'content':"""Vous êtes un assistant qui réponds en prenant le style d'un conteur pour enfants.
Toutes vos réponses doivent être d"une seule phrase."""},    
{'role':'user',
 'content':"""Ecrivez une histoire sur une carotte heureuse."""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Il était une fois, dans un jardin ensoleillé, une carotte joyeuse qui rêvait de raconter ses aventures aux autres légumes en dansant sous la douce brise du matin.


In [17]:
def get_completion_and_token_count(messages, 
                                   model="gpt-4o-mini", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message.content
    
    token_dict = {
'prompt_tokens':response.usage.prompt_tokens,
'completion_tokens':response.usage.completion_tokens,
'total_tokens':response.usage.total_tokens,
    }

    return content, token_dict

In [18]:
messages = [
{'role':'system', 
 'content':"""Vous êtes un assistant qui réponds en prenant le style d'un conteur pour enfants."""},    
{'role':'user',
 'content':"""Ecrivez une très court poème sur une carotte heureuse."""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [19]:
print(response)

Dans le jardin, sous le soleil doré,  
Une carotte dansait, toute émerveillée.  
Ses feuilles vertes, comme des bras,  
Lui murmuraient : "Regarde, c'est la joie, là-bas !"  

Elle rêvait de gâteaux, de soupes bien chaudes,  
D'un monde où les légumes sont tous en ode.  
Avec un sourire, elle poussait bien droit,  
Car la carotte heureuse, c'est un vrai petit roi ! 🌱🥕✨


In [20]:
print(token_dict)

{'prompt_tokens': 43, 'completion_tokens': 109, 'total_tokens': 152}


: 

#### Notes sur l'utilisation de l'API OpenAI en dehors de cette salle de classe.

Pour installer la bibliothèque Python OpenAI :
```
!pip install openai
```

La bibliothèque doit être configurée avec la clé secrète de votre compte, qui est disponible sur le [site web](https://platform.openai.com/account/api-keys). 

Vous pouvez soit la définir comme variable d'environnement `OPENAI_API_KEY` avant d'utiliser la bibliothèque :
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Ou, définir `openai.api_key` :

```
import openai
openai.api_key = "sk-..."
```

#### Remarque complémentaire
- Dans ce cours nous utiliserons les antislashs `\` pour que le texte rentre dans l'écran sans utiliser une nouvelle ligne '\n'.
- GPT-4 n'est pas vraiment affecté par l'insertion de caractères de nouvelle ligne ou non. Mais en travaillant avec des LLM en général, vous pouvez considérer si les caractères de nouvelle ligne dans votre prompt peuvent affecter les performances du modèle.